In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')

In [ ]:
train.head(10)

In [ ]:
train.describe()

In [ ]:
train.columns

In [ ]:
cols = ['country', 'store', 'product']
for i in cols:
    print('Value Count in {} is :\n {} \n \n'.format(i, train[i].value_counts()))

In [ ]:
train.isnull().sum()

In [ ]:
train.describe()

# Exploratory Data analysis

In [ ]:
train_df = train.drop(['row_id'], axis = 1)
test_df = test.drop(['row_id'], axis = 1)

In [ ]:
train_df['date'] = pd.to_datetime(train_df['date'])
train_df['year'] = train_df['date'].dt.year
train_df['month'] = train_df['date'].dt.month
train_df['day'] = train_df['date'].dt.day
train_df['weekday'] = train_df['date'].dt.weekday
train_df['week_of_year'] = train_df['date'].dt.isocalendar().week
train_df['day_of_year'] = train_df['date'].dt.dayofyear
train_df['quarter'] = train_df['date'].dt.quarter
train_df['is_weekend'] = train_df.date.dt.weekday>4
train_df.head()

In [ ]:
train_df.head()

In [ ]:
test_df['date'] = pd.to_datetime(test_df['date'])
test_df['year'] = test_df['date'].dt.year
test_df['month'] = test_df['date'].dt.month
test_df['day'] = test_df['date'].dt.day
test_df['weekday'] = test_df['date'].dt.weekday
test_df['week_of_year'] = test_df['date'].dt.isocalendar().week
test_df['day_of_year'] = test_df['date'].dt.dayofyear
test_df['quarter'] = test_df['date'].dt.quarter
test_df['is_weekend'] = test_df.date.dt.weekday>4
test_df.head()

In [ ]:
country = pd.DataFrame()
country = train_df.groupby('country').sum()
country.head()

In [ ]:
train_df = train_df.drop('date', axis = 1)
test_df = test_df.drop('date', axis = 1)

In [ ]:
def labelling(data, feature, plt):
    total_height = 0
    bar_plots = plt.patches
    for bars in bar_plots:
        total_height+=bars.get_height()
    for bars in bar_plots:
        percentage = '{:.1f}%'.format(100 * bars.get_height()/total_height)
        x = (bars.get_x() + bars.get_width()) / 2.0
        y = bars.get_height()
        plt.text(x,y+2, str(int(y))+','+percentage, ha='center', fontweight='bold', fontsize=1)
    

In [ ]:
import seaborn as sns
fig = plt.figure(figsize = (13,7))
sns.set_theme(style = 'whitegrid')
plot = sns.barplot(x = country.index, y= country['num_sold'], palette = 'rocket')
plt.xlabel('Country', fontweight = 'bold')
plt.ylabel('Number of sold items', fontweight = 'bold')
labelling(country,'country',plot)
plt.show()

can't understand what's wrong with the text in this. ill deal with later

## Country wise analysis

In [ ]:
finland = train_df[train_df['country'] == 'Finland']
finland

In [ ]:
finland_store = finland.groupby('store').sum()
finland_store.head()

modelling and processing

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder


In [ ]:
le = LabelEncoder()
col_encoder = ['country','store','product', 'is_weekend']
for i in col_encoder:
    train_df[i] = le.fit_transform(train_df[i])

In [ ]:
y = train_df['num_sold']
x = train_df.drop('num_sold', axis = 1)

In [ ]:
x['week_of_year'] = x['week_of_year'].astype(int)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

In [ ]:
print("Shape of x_train is :",x_train.shape)
print("Shape of x_test is :",x_test.shape)
print("Shape of y_train is :",y_train.shape)
print("Shape of y_test is :",y_test.shape)

In [ ]:
def SMAPE(y_true,y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    smape=0
    if(len(y_true)==len(y_pred)):
        smape = (100/len(y_true)) * np.sum(2* np.abs(y_pred-y_true)/(np.abs(y_true)+np.abs(y_pred)))
        smape_2 = (1/len(y_true))*np.sum(2*(np.abs(y_pred - y_true))/np.abs(y_true) + np.abs(y_pred))
    else:
        return
    return(smape)

In [ ]:
x_train.columns

In [ ]:
dic_model = {
    "RandomForestRegerssor":RandomForestRegressor(n_estimators = 100),
    'GradientBoosting':GradientBoostingRegressor(n_estimators=100),
    'XGradientBoosting':xgb.XGBRegressor(n_estimators=100),
    #'CatBoostRegressor':cb.CatBoostRegressor(n_estimators=100),
    'LightGBM': lgb.LGBMRegressor(n_estimators=100)
}
for i in dic_model:
    #Training
    print('Training with '+ i + ' Algorithm: \n')
    model = dic_model[i].fit(x_train,y_train)
    
    #Predicting
    print('Predicting with '+ i +' Algorithm: \n')
    prediction = model.predict(x_test)
    
    #Using SMAPE for prediction
    print('SMAPE of ' + i + ' Model is: ' + str(SMAPE(y_test, prediction)))
    print('---------------------------------------------------------------------')

Optimizing the XGBoost

In [ ]:
import optuna

def objective(trial, data = x, target =y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
    params = {
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02,0.2,1.0]),
        #'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        #'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
    }
    model = xgb.XGBRegressor(**params)
    model.fit(x_train,y_train)
    preds = model.predict(x_test)
    Smape = SMAPE(y_test, preds)
    return Smape

In [ ]:
study_xgb = optuna.create_study(direction = 'minimize')
study_xgb.optimize(objective, n_trials = 50)

In [ ]:
best_trial = study_xgb.best_trial
print(best_trial.value)
print(best_trial.params)

In [ ]:
model_xgb = xgb.XGBRegressor(**best_trial.params)
model_xgb.fit(x,y)

Preprocessing the test dataset

In [ ]:
test_df.columns

In [ ]:
#Preprocessing test dataset
le = LabelEncoder()
col_encode = ['country','store','product','is_weekend']
for i in col_encode:
    test_df[i] = le.fit_transform(test_df[i])

In [ ]:
test_df['week_of_year'] = test_df['week_of_year'].astype('int')

In [ ]:
test_df.head()

In [ ]:
test.head()

In [ ]:
pred_xgb = model_xgb.predict(test_df)

In [ ]:
dataframe = pd.DataFrame({"row_id":test['row_id'], 'num_sold':pred_xgb})
dataframe.head()

In [ ]:
dataframe.to_csv("Output_xgb_addedfeatures.csv",index=False)